In [ ]:
"""
RCSB Disulfide Bond Database Browser
Author: Eric G. Suchanek, PhD
Last revision: 11/2/2023
"""

import panel as pn
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import hvplot.pandas


pn.extension("plotly", "tabulator")
import proteusPy

from proteusPy.Disulfide import Disulfide
from proteusPy.DisulfideLoader import Load_PDB_SS
from proteusPy.DisulfideClasses import (
    plot_count_vs_classid,
    plot_count_vs_class_df,
)

_vers = 0.1

_default_binary = "00000"
_binarylist = [
    "00000",
    "00002",
    "00022",
    "00200",
    "00202",
    "00220",
    "00222",
    "02000",
    "02002",
    "02020",
    "02022",
    "02200",
    "02202",
    "02220",
    "20000",
    "20002",
    "20020",
    "20022",
    "20200",
    "20202",
    "20220",
    "20222",
    "22000",
    "22002",
    "22020",
    "22022",
    "22200",
    "22202",
    "22220",
    "22222",
]

PDB_SS = Load_PDB_SS(verbose=True, subset=False)

vers = PDB_SS.version
tot = PDB_SS.TotalDisulfides
pdbs = len(PDB_SS.SSDict)
orientation_widget = True
enable_keybindings = True

RCSB_list = sorted(PDB_SS.IDList)

# pn.state.template.param.update(title=f"RCSB Disulfide Class Browser: {tot:,} Disulfides, {pdbs:,} Structures, V{vers}")


def get_theme() -> str:
    """Return the current theme: 'default' or 'dark'

    Returns:
        str: The current theme
    """
    args = pn.state.session_args
    if "theme" in args and args["theme"][0] == b"dark":
        return "dark"
    return "light"


# Widgets

binary_class_widget = pn.widgets.Select(
    name="Binary Class", value=_default_binary, options=_binarylist
)


# markdown panels for various text outputs
title_md = pn.pane.Markdown("Title")
output_md = pn.pane.Markdown("Output goes here")
db_md = pn.pane.Markdown("Database Info goes here")

info_md = pn.pane.Markdown("SS Info")
ss_info = pn.WidgetBox("# Disulfide Info", info_md)
db_info = pn.Column("### RCSB Database Info", db_md)

# controls on sidebar
widgets = pn.WidgetBox("# Binary Class Selection", binary_class_widget)

# Callbacks

# rcsb_selector_widget.param.watch(get_ss_idlist, 'value')
# rcsb_ss_widget.param.watch(click_plot, 'value')
# styles_group.param.watch(click_plot, 'value')
# single_checkbox.param.watch(update_single, 'value')

In [14]:
def update_title(ss):
    src = ss.pdb_id
    name = ss.name

    title = f"## {name}"
    title_md.object = title


def update_info(ss):
    """Update the info for the input disulfide."""

    src = ss.pdb_id
    enrg = ss.energy
    name = ss.name
    resolution = ss.resolution

    info_string = f"### {name}  \n**Resolution:** {resolution:.2f} Å  \n**Energy:** {enrg:.2f} kcal/mol  \n**Cα distance:** {ss.ca_distance:.2f} Å  \n**Cβ distance:** {ss.cb_distance:.2f} Å  \n**Torsion Length:** {ss.torsion_length:.2f}°"
    info_md.object = info_string


@pn.depends(binary_class_widget.param.value)
def update_sixclass_info(binary_class, loader=PDB_SS):
    """Update the list of eight classes based on the binary class selected."""
    ss_list = loader.tclass.binary_to_six_class(class_id)

    info_string = f"### {ss_list}"
    print(f"Info: {info_string}")
    info_md.object = info_string


@pn.depends(binary_class_widget.param.value)
def plot_classes(class_id, loader=PDB_SS, theme="light"):
    ss_list = loader.tclass.binary_to_class(class_id, 8)
    df = PDB_SS.enumerate_class_fromlist(ss_list, 8)
    return df[class_id].hvplot()
    # fig = plot_six_classcount_vs_class_df(df, class_id, theme=theme)
    # return pn.pane.Plotly(fig)


def plot_six_classcount_vs_class_df(
    df, title="title", theme="light", save=False, savedir="."
):
    """
    Plots a line graph of count vs class ID using Plotly.

    :param df: A pandas DataFrame containing the data to be plotted.
    :param title: A string representing the title of the plot (default is 'title').
    :param theme: A string representing the name of the theme to use. Can be either 'notebook' or 'plotly_dark'. Default is 'plotly_dark'.
    :return: None
    """
    import hvplot.pandas

    fig = px.line(
        df,
        x="class_id",
        y="count",
        title=f"{title}",
        labels={"class_id": "Class ID", "count": "Count"},
    )

    if theme == "light":
        fig.update_layout(template="plotly_white")
    else:
        fig.update_layout(template="plotly_dark")

    fig.update_layout(
        showlegend=True,
        title_x=0.5,
        title_font=dict(size=20),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
    )
    fig.update_layout(autosize=True)

    if save:
        fname = f"{savedir}/{title}.png"
        fig.write_image(fname, "png")
    else:
        fig.show()
    return fig

In [15]:
LHSpiral_neg = PDB_SS.tclass.binary_to_six_class("00000")
RHSpiral_neg = PDB_SS.tclass.binary_to_six_class("02220")

In [18]:
# Create a Panel layout
layout = pn.Column(binary_class_widget, pn.Column(plot_classes))

# Show the Panel
layout.servable()

KeyError: '00000'

In [17]:
import panel as pn
import pandas as pd
import hvplot.pandas

# Create a DataFrame
df = pd.DataFrame({"A": range(1, 6), "B": range(10, 60, 10)})

# Create a dropdown widget
dropdown = pn.widgets.Select(name="Select column", options=list(df.columns))


# Create a function to update the plot based on the selected column
@pn.depends(dropdown.param.value)
def update_plot(column):
    return df[column].hvplot()


# Create a Panel layout
layout = pn.Column(dropdown, update_plot)

# Show the Panel
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'121b68a4-ad16-47f6-a762-c920ae825182': {'version…